# Imports

In [5]:
# IMPORTS
import copy
import numpy as np
from enum import Enum
import bisect 
import time

# CLASS INITIALIZATIONS

In [6]:
class Direction(Enum):
  HORIZONTAL = 1
  VERTICAL = 2

class Coordinate():
  def __init__(self, X, Y):
    self.X = X 
    self.Y = Y
  # Primarily for testing to see if Coordinate works
  def __str__(self):
    return "(" + str(self.X) + ", " + str(self.Y) + ")"

#
# CAR
# 

class Car(object):
  def __init__(self, letter, coordinate, fuel):
    self.letter = letter  # Remember that A is the most important
    self.coordinates = []
    self.coordinates.append(coordinate)
    self.fuel = int(fuel)

  def isA(self):
    return self.letter == 'A'

  def move(self, amt):
    if self.direction == Direction.HORIZONTAL:
      for c in self.coordinates:
        c.Y += amt
    else:
      for c in self.coordinates:
        c.X += amt
    self.fuel -= abs(amt)
  
  def canMove(self):
    return self.fuel > 0

  def compareLetters(self, letter):
    return self.letter == letter

  def addCoordinate(self, newCoord):
    self.coordinates.append(newCoord)
    # Update direction
    # L-R (Horizontal)
    if self.coordinates[0].X == newCoord.X and self.coordinates[0].Y != newCoord.Y:
      self.direction = Direction.HORIZONTAL
    # U-D (Vertical)
    else:
      self.direction = Direction.VERTICAL

  def getLetter(self):
    return self.letter; 

  # Primarily for testing to see if Car works
  def __str__(self):
    s = self.letter + " -> ";
    for i in self.coordinates:
      s += str(i)
    return s + " | " + str(self.fuel) + " | " + str(self.direction)

  @staticmethod
  def create(letter, coordinates, fuel):
    return Car(letter, coordinates, fuel)


#
# BOARD
# 

class Board(object):
  def __init__(self, cars):
    self.grid = [['.' for i in range(6)] for j in range(6)]
    self.size = {'X': 6, 'Y': 6}
    self.cars = []
    for car in cars:  
      c = car
      self.cars.append(copy.deepcopy(c));
      for coord in c.coordinates:
        self.grid[coord.X][coord.Y] = car.letter
        
  def __eq__(self, obj):
        return self.grid == obj.grid

  def findCarPaths(self, car):
    if car.canMove():
      fwd = self.checkPathForward(car)
      bwd = self.checkPathBackward(car)
      carPath = bwd + fwd
    else:
      carPath = []
    return carPath
  
  def moveCar(self, carLetter, amt):
    for car in self.cars: 
      if car.letter == carLetter:
        for coord in car.coordinates:
          #print(coord)
          self.grid[coord.X][coord.Y] = "."
        car.move(amt)
        for coord in car.coordinates:
          #print(coord)
          self.grid[coord.X][coord.Y] = carLetter
        
        # Is this a regular car that just left?
        if car.letter == self.grid[2][5] and car.direction == Direction.HORIZONTAL and car.letter != "A":
          for coord in car.coordinates:
            self.grid[coord.X][coord.Y] = "."
          self.cars.remove(car)  
        
        
        direction="";
        if(amt<0 and car.direction==Direction.VERTICAL):
            direction="up"
        if(amt>0 and car.direction==Direction.VERTICAL):
            direction="down"
        if(amt<0 and car.direction==Direction.HORIZONTAL):
            direction="left"
        if(amt>0 and car.direction==Direction.HORIZONTAL):
            direction="right"        
        #returns the move performed
        return((carLetter, direction, abs(amt), car.fuel, self.stringLine()))
        
  # ......
  # ...AA. (1,3)(1,4)
  # ......
  # startPoint = 4
  # range(5,6) -> 5
  # next gridPos = (1,5)
  # append -> 5 - 4 = 1

  # ...D.. (0,3)
  # ...D.. (1,3)
  # ......
  # startPoint = 1
  # range(2,6) -> 2, 3, 4, 5
  # next gridPos = (2,3)
  # append -> 2 - 1 = 1
  def checkPathForward(self, car):
    dir = car.direction
    lastCoord = car.coordinates[len(car.coordinates)-1]
    if dir == Direction.HORIZONTAL:
      startPoint = lastCoord.Y
    else:
      startPoint = lastCoord.X

    possibleMoves = []
    for i in range(startPoint+1, 6):
        #fuel req
      if(car.fuel < i-startPoint):
        continue;
      # Find next position to check    
      if dir == Direction.HORIZONTAL:
        gridPos = self.grid[lastCoord.X][i] 
      else:
        gridPos = self.grid[i][lastCoord.Y]

      # If there's an open position, add and continue
      # Else, there is something blocking the way and there's no way to continue any more
      if gridPos == ".":
        possibleMoves.append(i-startPoint)
      else:
        break;

    return possibleMoves

  # ......
  # ...AA. (1,3)(1,4)
  # ......
  # startPoint = 3
  # range(3,0,-1) -> 3, 2, 1
  # i = i - 1 -> 2, 1, 0
  # next gridPos = (1,2)
  # append -> 2 - 3 = -1

  # ...D.. (0,3)
  # ...D.. (1,3)
  # ......
  # startPoint = 0
  # range(0,0) -> NONE
  # next gridPos = NONE
  def checkPathBackward(self, car):
    dir = car.direction
    lastCoord = car.coordinates[0]
    if dir == Direction.HORIZONTAL:
      startPoint = lastCoord.Y
    else:
      startPoint = lastCoord.X

    possibleMoves = []
    for i in range(startPoint, 0, -1):
      if (car.fuel <= (startPoint-i)):
        continue
      # Find next position to check
      if dir == Direction.HORIZONTAL:
        gridPos = self.grid[lastCoord.X][i-1] 
      else:
        gridPos = self.grid[i-1][lastCoord.Y]

      # If there's an open position, add and continue
      # Else, there is something blocking the way and there's no way to continue any more
      if gridPos == ".":
        possibleMoves.append(i-startPoint-1)
      else:
        break;

    return possibleMoves

  def winConditionMet(self):
    if self.grid[2][5] == "A":
      return True      
    return False

  def __str__(self): 
    s = ""
    for x in range(6):
      for y in range(6):
        s += self.grid[x][y]
      s += "\n" 
    return s

  def stringLine(self):
    s = ""
    for x in range(6):
      for y in range(6):
        s += self.grid[x][y]
    return s

  def AllUsedCars(self):
    s = ""
    for c in cars:
      if(c.fuel < 100):
        s += c.getLetter + " " + c.fuel + " ";
    return s


  #Heuristic definitions:

  #h1 = The number of blocking vehicles
  def h1(self):
        count = 0
        last = " "
        for i in range(6):
            e = self.grid[2][5-i]
            if e == "A":
                break
            elif e == last:
                continue
            elif e != ".":
                count += 1
                last = e
        return count
  
  #h2 = The number of blocked positions
  def h2(self):
        count = 0
        for i in range(6):
            e = self.grid[2][5-i]
            if e == "A":
                break
            elif e != ".":
                count += 1
        return count
    
  #h3 = The number of blocking vehicles (h1) multiplied by a constant
  def h3(self, constant):
        return self.h1()*constant
    
  #h4 = 
  def h4(self):
        count = 0
        last = " "
        blocked = []
        for i in range(6):
            e = self.grid[2][5-i]
            if e == "A":
                break
            elif e == last:
                continue
            elif e != ".":
                count += 1
                last = e
                #Check if car is horizontal by looking at the next slot in the grid
                if self.grid[2][5-i-1] == e:
                    continue
                #If the car is not horizontal, we count how many squares block it in both directions
                else:
                    last2 = " "
                    up = 0
                    for j in range(2):
                        if self.grid[2-j-1][5-i] == e or self.grid[2-j-1][5-i] == "." or self.grid[2-j-1][5-i] == last2:
                            continue
                        else:
                            up += 1
                            last2 = self.grid[2-j-1][5-i]
                    if up == 0:
                        continue
                    last2 = " "
                    down = 0
                    for j in range(3):
                        if self.grid[2+j+1][5-i] == e or self.grid[2+j+1][5-i] == "." or self.grid[2+j+1][5-i] == last2:
                            continue
                        else:
                            down += 1
                            last2 = self.grid[2+j+1][5-i]
                    if down == 0:
                        continue
                    blocked.append(min(up, down))
                        
        if blocked:
            return count + min(blocked)
        else:
            return count
                    
                
        
        if self.grid[2][5] != "." and self.grid[2][5] != "A":
            return 1
        else:
            return 0

# Extracting Data From Sample-input.txt

In [74]:
demos = []
demoFuels = []

with open('Sample\\sample-input-50.txt') as f:
    for line in f:
      if not line.startswith(('#', '\n')):
        in_arr = np.array([line.strip()])
        data = np.char.split(in_arr)[0]
        demos.append(data)
f.close()

print(demos)

[['Z.BOOOZ.B....AAC...D.C...DPPPE.....E'], ['....ZZ....O..AA.OB....OB.....C...DDC'], ['....Z.....Z.AA.BC..O.BC..ODDEE.O....'], ['..ZOOO..ZP.B.AAP.B...P.......C.....C'], ['ZZ..O...BBOC..AAOC...DD...EFF...E...'], ['...O.....O.Z.AAO.Z......B.C...B.CPPP'], ['.....Z..OOOZAA.B.C...B.C.DDPPP......'], ['ZOOO..Z.BC.DAABC.D..PPP.............'], ['ZZBBCC....DE..AADE..O.....OFGG..OF..'], ['ZZ.BCCOOOB.D..EAADFFEGGHIJJ..HI...KK'], ['...ZZ....BCCAA.B.D...E.D...EFF......'], ['OZBBCCOZ.DEEOAADF.GGH.FP..HIIP.JJKKP'], ['OZBCCDOZBEFDOAAEF....GHHII.G..JJPPP.'], ['Z.BOOOZ.BCDDPAACEFPG..EFPGQQQH.....H'], ['...OOO....P...AAP..QQQP...ZBCC..ZB..'], ['Z.O...Z.O...AAO..........BBC.DPPPC.D'], ['..OPPP..OZ.QAAOZBQ...RBQ.CCRD....RD.'], ['ZZ..O.BCDDOEBCAAOE.P.....PF.GG.PFQQQ'], ['..Z.BB..ZC...AACD.....D...E.FF..E...'], ['...OOO......AA.P..ZZ.PB..C.PB..C.QQQ'], ['..ZBCC..ZB....AADE....DE.FFOOO......'], ['.ZZ.BB...CC.AA.OD....OD....OEE......'], ['OOO.P...ZZP...AAP.......BBC.DD..C...'], ['ZBBCDDZ..CEEAA..O..PPPOF..G.OFH

# Setting the boards up

In [75]:
boards = []
for data in demos:
  # Seperate Grid Data from the Fuel Parameters
  grid = data[0];
  fuels = []
  if len(data) > 1:
    fuels.append(data[1:])

  cars = []
  for row in range(6):
    for col in range(6):
      current = grid[row*6 + col];
      
      # If Unique: Create a car and append
      # If Not Unique: Concat to same letter
      if current == ".":
        continue

      unique = True
      for c in cars:
        if c.compareLetters(current):
          c.addCoordinate(Coordinate(row,col))
          unique = False

      if unique:
        # Now check if there's a unique fuel amount or if its the default 100
        fuelAmt = 100
        if len(fuels) > 0:
          for f in fuels[0]:
            if current == f[0]:
              fuelAmt = f[1:]

        cars.append(Car.create(current, Coordinate(row,col), fuelAmt));

  # Checking if all cars were made properly
  print("CARS: ")
  print("--------")
  for c in cars:
    print(c)
  print()
  
  board = Board(cars)
  # Checking if all boards were made properly
  print("BOARD: ")
  print("--------")
  print(board)
  print()

  boards.append(board)


  for b in boards:
    for c in b.cars:
      print(c)
    print()

CARS: 
--------
Z -> (0, 0)(1, 0) | 100 | Direction.VERTICAL
B -> (0, 2)(1, 2) | 100 | Direction.VERTICAL
O -> (0, 3)(0, 4)(0, 5) | 100 | Direction.HORIZONTAL
A -> (2, 1)(2, 2) | 100 | Direction.HORIZONTAL
C -> (2, 3)(3, 3) | 100 | Direction.VERTICAL
D -> (3, 1)(4, 1) | 100 | Direction.VERTICAL
P -> (4, 2)(4, 3)(4, 4) | 100 | Direction.HORIZONTAL
E -> (4, 5)(5, 5) | 100 | Direction.VERTICAL

BOARD: 
--------
Z.BOOO
Z.B...
.AAC..
.D.C..
.DPPPE
.....E


Z -> (0, 0)(1, 0) | 100 | Direction.VERTICAL
B -> (0, 2)(1, 2) | 100 | Direction.VERTICAL
O -> (0, 3)(0, 4)(0, 5) | 100 | Direction.HORIZONTAL
A -> (2, 1)(2, 2) | 100 | Direction.HORIZONTAL
C -> (2, 3)(3, 3) | 100 | Direction.VERTICAL
D -> (3, 1)(4, 1) | 100 | Direction.VERTICAL
P -> (4, 2)(4, 3)(4, 4) | 100 | Direction.HORIZONTAL
E -> (4, 5)(5, 5) | 100 | Direction.VERTICAL

CARS: 
--------
Z -> (0, 4)(0, 5) | 100 | Direction.HORIZONTAL
O -> (1, 4)(2, 4)(3, 4) | 100 | Direction.VERTICAL
A -> (2, 1)(2, 2) | 100 | Direction.HORIZONTAL
B ->

O -> (0, 2)(1, 2)(2, 2) | 100 | Direction.VERTICAL
A -> (2, 0)(2, 1) | 100 | Direction.HORIZONTAL
B -> (4, 1)(4, 2) | 100 | Direction.HORIZONTAL
C -> (4, 3)(5, 3) | 100 | Direction.VERTICAL
D -> (4, 5)(5, 5) | 100 | Direction.VERTICAL
P -> (5, 0)(5, 1)(5, 2) | 100 | Direction.HORIZONTAL

O -> (0, 2)(1, 2)(2, 2) | 100 | Direction.VERTICAL
P -> (0, 3)(0, 4)(0, 5) | 100 | Direction.HORIZONTAL
Z -> (1, 3)(2, 3) | 100 | Direction.VERTICAL
Q -> (1, 5)(2, 5)(3, 5) | 100 | Direction.VERTICAL
A -> (2, 0)(2, 1) | 100 | Direction.HORIZONTAL
B -> (2, 4)(3, 4) | 100 | Direction.VERTICAL
R -> (3, 3)(4, 3)(5, 3) | 100 | Direction.VERTICAL
C -> (4, 1)(4, 2) | 100 | Direction.HORIZONTAL
D -> (4, 4)(5, 4) | 100 | Direction.VERTICAL

Z -> (0, 0)(0, 1) | 100 | Direction.HORIZONTAL
O -> (0, 4)(1, 4)(2, 4) | 100 | Direction.VERTICAL
B -> (1, 0)(2, 0) | 100 | Direction.VERTICAL
C -> (1, 1)(2, 1) | 100 | Direction.VERTICAL
D -> (1, 2)(1, 3) | 100 | Direction.HORIZONTAL
E -> (1, 5)(2, 5) | 100 | Direction.VERT

A -> (2, 0)(2, 1) | 100 | Direction.HORIZONTAL
B -> (2, 4)(3, 4) | 100 | Direction.VERTICAL
R -> (3, 3)(4, 3)(5, 3) | 100 | Direction.VERTICAL
C -> (4, 1)(4, 2) | 100 | Direction.HORIZONTAL
D -> (4, 4)(5, 4) | 100 | Direction.VERTICAL

Z -> (0, 0)(0, 1) | 100 | Direction.HORIZONTAL
O -> (0, 4)(1, 4)(2, 4) | 100 | Direction.VERTICAL
B -> (1, 0)(2, 0) | 100 | Direction.VERTICAL
C -> (1, 1)(2, 1) | 100 | Direction.VERTICAL
D -> (1, 2)(1, 3) | 100 | Direction.HORIZONTAL
E -> (1, 5)(2, 5) | 100 | Direction.VERTICAL
A -> (2, 2)(2, 3) | 100 | Direction.HORIZONTAL
P -> (3, 1)(4, 1)(5, 1) | 100 | Direction.VERTICAL
F -> (4, 2)(5, 2) | 100 | Direction.VERTICAL
G -> (4, 4)(4, 5) | 100 | Direction.HORIZONTAL
Q -> (5, 3)(5, 4)(5, 5) | 100 | Direction.HORIZONTAL

Z -> (0, 2)(1, 2) | 100 | Direction.VERTICAL
B -> (0, 4)(0, 5) | 100 | Direction.HORIZONTAL
C -> (1, 3)(2, 3) | 100 | Direction.VERTICAL
A -> (2, 1)(2, 2) | 100 | Direction.HORIZONTAL
D -> (2, 4)(3, 4) | 100 | Direction.VERTICAL
E -> (4, 2)

C -> (2, 4)(3, 4) | 100 | Direction.VERTICAL
D -> (2, 5)(3, 5) | 100 | Direction.VERTICAL
E -> (5, 2)(5, 3) | 100 | Direction.HORIZONTAL

O -> (0, 3)(0, 4)(0, 5) | 100 | Direction.HORIZONTAL
Z -> (1, 4)(1, 5) | 100 | Direction.HORIZONTAL
B -> (2, 0)(2, 1) | 100 | Direction.HORIZONTAL
P -> (2, 2)(3, 2)(4, 2) | 100 | Direction.VERTICAL
A -> (2, 3)(2, 4) | 100 | Direction.HORIZONTAL
C -> (2, 5)(3, 5) | 100 | Direction.VERTICAL
Q -> (4, 3)(4, 4)(4, 5) | 100 | Direction.HORIZONTAL

Z -> (0, 0)(1, 0) | 100 | Direction.VERTICAL
B -> (0, 1)(0, 2) | 100 | Direction.HORIZONTAL
C -> (0, 4)(1, 4) | 100 | Direction.VERTICAL
D -> (1, 2)(2, 2) | 100 | Direction.VERTICAL
O -> (1, 5)(2, 5)(3, 5) | 100 | Direction.VERTICAL
A -> (2, 0)(2, 1) | 100 | Direction.HORIZONTAL
P -> (3, 1)(4, 1)(5, 1) | 100 | Direction.VERTICAL
Q -> (3, 2)(4, 2)(5, 2) | 100 | Direction.VERTICAL
E -> (3, 3)(3, 4) | 100 | Direction.HORIZONTAL
F -> (4, 3)(5, 3) | 100 | Direction.VERTICAL
G -> (4, 4)(4, 5) | 100 | Direction.HORIZONT

P -> (5, 1)(5, 2)(5, 3) | 100 | Direction.HORIZONTAL

BOARD: 
--------
ZZOOO.
....B.
..AAB.
..C.DD
..C.EE
.PPP..


Z -> (0, 0)(1, 0) | 100 | Direction.VERTICAL
B -> (0, 2)(1, 2) | 100 | Direction.VERTICAL
O -> (0, 3)(0, 4)(0, 5) | 100 | Direction.HORIZONTAL
A -> (2, 1)(2, 2) | 100 | Direction.HORIZONTAL
C -> (2, 3)(3, 3) | 100 | Direction.VERTICAL
D -> (3, 1)(4, 1) | 100 | Direction.VERTICAL
P -> (4, 2)(4, 3)(4, 4) | 100 | Direction.HORIZONTAL
E -> (4, 5)(5, 5) | 100 | Direction.VERTICAL

Z -> (0, 4)(0, 5) | 100 | Direction.HORIZONTAL
O -> (1, 4)(2, 4)(3, 4) | 100 | Direction.VERTICAL
A -> (2, 1)(2, 2) | 100 | Direction.HORIZONTAL
B -> (2, 5)(3, 5) | 100 | Direction.VERTICAL
C -> (4, 5)(5, 5) | 100 | Direction.VERTICAL
D -> (5, 3)(5, 4) | 100 | Direction.HORIZONTAL

Z -> (0, 4)(1, 4) | 100 | Direction.VERTICAL
A -> (2, 0)(2, 1) | 100 | Direction.HORIZONTAL
B -> (2, 3)(3, 3) | 100 | Direction.VERTICAL
C -> (2, 4)(3, 4) | 100 | Direction.VERTICAL
O -> (3, 1)(4, 1)(5, 1) | 100 | Direction

J -> (5, 1)(5, 2) | 100 | Direction.HORIZONTAL
K -> (5, 3)(5, 4) | 100 | Direction.HORIZONTAL

O -> (0, 0)(1, 0)(2, 0) | 100 | Direction.VERTICAL
Z -> (0, 1)(1, 1) | 100 | Direction.VERTICAL
B -> (0, 2)(1, 2) | 100 | Direction.VERTICAL
C -> (0, 3)(0, 4) | 100 | Direction.HORIZONTAL
D -> (0, 5)(1, 5) | 100 | Direction.VERTICAL
E -> (1, 3)(2, 3) | 100 | Direction.VERTICAL
F -> (1, 4)(2, 4) | 100 | Direction.VERTICAL
A -> (2, 1)(2, 2) | 100 | Direction.HORIZONTAL
G -> (3, 3)(4, 3) | 100 | Direction.VERTICAL
H -> (3, 4)(3, 5) | 100 | Direction.HORIZONTAL
I -> (4, 0)(4, 1) | 100 | Direction.HORIZONTAL
J -> (5, 0)(5, 1) | 100 | Direction.HORIZONTAL
P -> (5, 2)(5, 3)(5, 4) | 100 | Direction.HORIZONTAL

Z -> (0, 0)(1, 0) | 100 | Direction.VERTICAL
B -> (0, 2)(1, 2) | 100 | Direction.VERTICAL
O -> (0, 3)(0, 4)(0, 5) | 100 | Direction.HORIZONTAL
C -> (1, 3)(2, 3) | 100 | Direction.VERTICAL
D -> (1, 4)(1, 5) | 100 | Direction.HORIZONTAL
P -> (2, 0)(3, 0)(4, 0) | 100 | Direction.VERTICAL
A -> (2, 

D -> (2, 5)(3, 5) | 100 | Direction.VERTICAL
E -> (3, 3)(4, 3) | 100 | Direction.VERTICAL
F -> (4, 4)(4, 5) | 100 | Direction.HORIZONTAL

O -> (0, 0)(1, 0)(2, 0) | 100 | Direction.VERTICAL
Z -> (0, 1)(1, 1) | 100 | Direction.VERTICAL
B -> (0, 2)(0, 3) | 100 | Direction.HORIZONTAL
C -> (0, 4)(0, 5) | 100 | Direction.HORIZONTAL
D -> (1, 3)(2, 3) | 100 | Direction.VERTICAL
E -> (1, 4)(1, 5) | 100 | Direction.HORIZONTAL
A -> (2, 1)(2, 2) | 100 | Direction.HORIZONTAL
F -> (2, 4)(3, 4) | 100 | Direction.VERTICAL
G -> (3, 0)(3, 1) | 100 | Direction.HORIZONTAL
H -> (3, 2)(4, 2) | 100 | Direction.VERTICAL
P -> (3, 5)(4, 5)(5, 5) | 100 | Direction.VERTICAL
I -> (4, 3)(4, 4) | 100 | Direction.HORIZONTAL
J -> (5, 1)(5, 2) | 100 | Direction.HORIZONTAL
K -> (5, 3)(5, 4) | 100 | Direction.HORIZONTAL

O -> (0, 0)(1, 0)(2, 0) | 100 | Direction.VERTICAL
Z -> (0, 1)(1, 1) | 100 | Direction.VERTICAL
B -> (0, 2)(1, 2) | 100 | Direction.VERTICAL
C -> (0, 3)(0, 4) | 100 | Direction.HORIZONTAL
D -> (0, 5)(1, 

C -> (4, 2)(5, 2) | 100 | Direction.VERTICAL
P -> (5, 3)(5, 4)(5, 5) | 100 | Direction.HORIZONTAL

Z -> (0, 5)(1, 5) | 100 | Direction.VERTICAL
O -> (1, 2)(1, 3)(1, 4) | 100 | Direction.HORIZONTAL
A -> (2, 0)(2, 1) | 100 | Direction.HORIZONTAL
B -> (2, 3)(3, 3) | 100 | Direction.VERTICAL
C -> (2, 5)(3, 5) | 100 | Direction.VERTICAL
D -> (4, 1)(4, 2) | 100 | Direction.HORIZONTAL
P -> (4, 3)(4, 4)(4, 5) | 100 | Direction.HORIZONTAL

Z -> (0, 0)(1, 0) | 100 | Direction.VERTICAL
O -> (0, 1)(0, 2)(0, 3) | 100 | Direction.HORIZONTAL
B -> (1, 2)(2, 2) | 100 | Direction.VERTICAL
C -> (1, 3)(2, 3) | 100 | Direction.VERTICAL
D -> (1, 5)(2, 5) | 100 | Direction.VERTICAL
A -> (2, 0)(2, 1) | 100 | Direction.HORIZONTAL
P -> (3, 2)(3, 3)(3, 4) | 100 | Direction.HORIZONTAL

Z -> (0, 0)(0, 1) | 100 | Direction.HORIZONTAL
B -> (0, 2)(0, 3) | 100 | Direction.HORIZONTAL
C -> (0, 4)(0, 5) | 100 | Direction.HORIZONTAL
D -> (1, 4)(2, 4) | 100 | Direction.VERTICAL
E -> (1, 5)(2, 5) | 100 | Direction.VERTICAL


# Building State Space Search - Uniform Cost Search (UCS)

In [59]:
class Node(object):
    def __init__(self, board, cost):
        self.board = board
        self.cost = cost
        self.path = []
        self.parent = 0
        self.search = ""
    def __lt__(self, other):
        return self.cost < other.cost
    def __eq__(self, other):
        return self.board == other.board
  
    
class UCS(object):
    def __init__(self):
        self.opened = []
        self.closed = []
        self.count = 0

    def recursivePrinting(self, node):
        if node.cost != 0:
            self.recursivePrinting(node.parent)  
        print(node.board)

    def search(self, board):
        self.count += 1
        #Initialise first Node with cost 0 (different for other algorithms)
        a = Node(copy.deepcopy(board), 0)
        self.opened.append(a)
        a.search = "0 0 0 " + a.board.stringLine()

        start_time = time.time()
        statessearched = 0
        
        f = open("Results\\ucs-search-"+ str(self.count) +".txt", "w")

        while(1):              
            #If open list is empty, exit
            if(len(self.opened)==0):
                print("No Solution")
                #Close last file
                f.close()
                #Write everything to file
                f = open("Results\\ucs-sol-"+ str(self.count) +".txt", "w")
                f.write("--------------------------------------------------------------------------------\n")
                f.write("Initial board configuration: " + board.stringLine())
                f.write("\n\n!\n" + str(board) + "\n")
                f.write("Car Fuel Available: ")
                for i in range(len(board.cars)):
                    f.write(""+board.cars[i].letter+":"+str(board.cars[i].fuel))
                    if i < len(board.cars) - 1:
                        f.write(", ")
                f.write("\n\n Sorry, could not solve the puzzle as specified.\nError: no solution found\n")
                runtime = time.time() - start_time
                f.write("Runtime: " + str(runtime) + " seconds")
                f.write("--------------------------------------------------------------------------------")
                f.close()
                
                f = open("Results\\all_results.txt", "a")
                f.write(str(self.count) + "\t\t\tUCS\t\tNA\t\tNA\t\t\t\t" + str(statessearched) + "\t\t\t\t" + str("%5.2f" % runtime) + "\n")
                f.close
                
                return

            #Pop the best candidate node
            curr = self.opened.pop(0)
            
            #print(curr.search)
            f.write(curr.search + "\n")

            #If the popped node is the solution, exit out and print stuff (later to file)
            if(curr.board.winConditionMet()):
                print("Initial board configuration")
                print()
                print(board)
                print("Car Fuel Available: ", end = '')
                for i in range(len(board.cars)):
                    print(""+board.cars[i].letter+":"+str(board.cars[i].fuel)+", ", end ='')
                print()

                print("Runtime: %s seconds" % (time.time() - start_time))
                print("Number of moves: ", curr.cost)
                print("Search path length: %s states" % statessearched)

                print("------")
                #self.recursivePrinting(curr)
                print("Solution path: ")
                for move in curr.path:
                    print(move[0]+ " "+move[1] + " " + str(move[2])+"; ", end='')

                ######

                #Close last file
                f.close()
                #Write everything to file
                f = open("Results\\ucs-sol-"+ str(self.count) +".txt", "w")
                f.write("Initial board configuration: " + board.stringLine())
                f.write("\n\n" + str(board) + "\n")
                f.write("Car Fuel Available: ")
                for i in range(len(board.cars)):
                    f.write(""+board.cars[i].letter+":"+str(board.cars[i].fuel))
                    if i < len(board.cars) - 1:
                        f.write(", ")
                f.write("\n\n")
                runtime = time.time() - start_time
                f.write("Runtime: " + str(runtime) + " seconds")
                f.write("\nNumber of moves: " + str(curr.cost))
                f.write("\nSearch path length: " + str(statessearched) + " states")

                # Recursive here....
                f.write("\nSolution path: ")
                for move in curr.path:
                    f.write(move[0]+ " "+move[1] + " " + str(move[2])+"; ")

                f.write("\n\n");
                for move in curr.path:
                    f.write(move[0]+ "\t"+move[1] + "\t" + str(move[2]) + "\t" + str(move[3]) + "\t" + str(move[4])+"\n")

                f.write("\n" + str(curr.board) + "\n")
                f.close()
                
                f = open("Results\\all_results.txt", "a")
                f.write(str(self.count) + "\t\t\tUCS\t\tNA\t\t" + str(curr.cost) + "\t\t\t\t" + str(statessearched) + "\t\t\t\t" + str("%5.2f" % runtime) + "\n")
                f.close

                return

            #If it's not a solution insert into closed list and increment counter
            self.closed.append(curr)
            statessearched+=1
            #print()
            for c in curr.board.cars:
                paths = curr.board.findCarPaths(c)    
                if len(paths) > 0:  
                    #print("Car ", c.letter, " can move:", paths)

                    for i in range(len(paths)):
                        #Create a new node for successor
                        #Increase the cost of this new node (different in other algorithms)
                        newnode = Node(Board(curr.board.cars), curr.cost+1)   
                        newnode.parent = curr
                        newnode.path = copy.deepcopy(newnode.parent.path)

                        #Apply the move and store it
                        move = newnode.board.moveCar(c.letter, paths[i])
                        newnode.path.append(move)   
                        newnode.search = str(newnode.cost) + " " + str(newnode.cost) + " 0 " + move[4]

                        #If the item is already in the open list, skip it
                        #In other algos, need to check that cost is less
                        #For UCS, this will never be the case, since all costs are 1 in this case (breadth first search)
                        if newnode in self.opened or newnode in self.closed:
                            continue

                        #Insert into open list (in sorted order)
                        #bisect.insort(self.opened, newnode)
                        self.opened.append(newnode)



# Building State Space Search - Greedy Best First Search (GBFS)

In [60]:
class HNode(object):
    def __init__(self, board, cost, parentCost):
        self.board = board
        self.cost = cost
        self.hCost = 0
        self.path = []
        self.parent = 0
        self.parentCost = parentCost
        self.search = ""
    def __lt__(self, other):
        return self.cost < other.cost
    def __eq__(self, other):
        return self.board == other.board
    
    def addHeuristic(self, heuristic):
        if heuristic == 1:
            self.cost = self.cost + self.board.h1()
        elif heuristic == 2:
            self.cost = self.cost + self.board.h2()
        elif heuristic == 3:
            self.cost = self.cost + self.board.h3(3)
        elif heuristic == 4:
            self.cost = self.cost + self.board.h4()
        return self.cost
    def setCost(self, cost):
        self.cost = cost
        
        
class GBFS(object):
    def __init__(self):
        self.opened = []
        self.closed = []
        self.count = 0
        
    def next(self):
        self.count += 1
    
    def recursivePrinting(self, node):
        if node.cost != 0:
            self.recursivePrinting(node.parent)  
        print(node.board)
        
    def search(self, board, heuristic):
        
        #Initialise first Node with cost 0
        a = HNode(copy.deepcopy(board), 0, -1)
        self.opened.append(a)
        a.addHeuristic(heuristic)
        a.search = str(a.cost) + " 0 " + str(a.cost) + " " + a.board.stringLine()
        
        start_time = time.time()
        statessearched = 0
        
        f = open("Results\\gbfs-h"+str(heuristic)+"-search-"+ str(self.count) +".txt", "w")
      
        while(1):              
            #If open list is empty, exit
            if(len(self.opened)==0):
                print("No Solution")
                #Close last file
                f.close()
                #Write everything to file
                f = open("Results\\gbfs-h"+str(heuristic)+"-sol-"+ str(self.count) +".txt", "w")
                f.write("--------------------------------------------------------------------------------\n")
                f.write("Initial board configuration: " + board.stringLine())
                f.write("\n\n!\n" + str(board) + "\n")
                f.write("Car Fuel Available: ")
                for i in range(len(board.cars)):
                    f.write(""+board.cars[i].letter+":"+str(board.cars[i].fuel))
                    if i < len(board.cars) - 1:
                        f.write(", ")
                f.write("\n\n Sorry, could not solve the puzzle as specified.\nError: no solution found\n")
                runtime = time.time() - start_time
                f.write("Runtime: " + str(runtime) + " seconds")
                f.write("--------------------------------------------------------------------------------")
                f.close()
                
                f = open("Results\\all_results.txt", "a")
                f.write(str(self.count) + "\t\t\tGBFS\t\th" + str(heuristic) + "\t\tNA\t\t\t\t" + str(statessearched) + "\t\t\t\t" + str("%5.2f" % runtime) + "\n")
                f.close
                
                return
        
            #Pop the best candidate node
            curr = self.opened.pop(0)
            curr.setCost(curr.parentCost + 1)
            
            #print(curr.search)
            f.write(curr.search + "\n")
        
            #If the popped node is the solution, exit out and print stuff (later to file)
            if(curr.board.winConditionMet()):
                print("Initial board configuration")
                print()
                print(board)
                print("Car Fuel Available: ", end = '')
                for i in range(len(board.cars)):
                    print(""+board.cars[i].letter+":"+str(board.cars[i].fuel)+", ", end ='')
                print()
                
                print("Runtime: %s seconds" % (time.time() - start_time))
                print("Number of moves: ", curr.cost)
                print("Search path length: %s states" % statessearched)

                print("------")
                #self.recursivePrinting(curr)
                print("Solution path: ")
                for move in curr.path:
                    print(move[0]+ " "+move[1] + " " + str(move[2])+"; ", end='')
                    
                ######
                #Close last file
                f.close()
                #Write everything to file
                f = open("Results\\gbfs-h"+str(heuristic)+"-sol-"+ str(self.count) +".txt", "w")
                f.write("Initial board configuration: " + board.stringLine())
                f.write("\n\n" + str(board) + "\n")
                f.write("Car Fuel Available: ")
                for i in range(len(board.cars)):
                    f.write(""+board.cars[i].letter+":"+str(board.cars[i].fuel))
                    if i < len(board.cars) - 1:
                        f.write(", ")
                f.write("\n\n")
                runtime = time.time() - start_time
                f.write("Runtime: " + str(runtime) + " seconds")
                f.write("\nNumber of moves: " + str(curr.cost))
                f.write("\nSearch path length: " + str(statessearched) + " states")

                # Recursive here....
                f.write("\nSolution path: ")
                for move in curr.path:
                    f.write(move[0]+ " "+move[1] + " " + str(move[2])+"; ")

                f.write("\n\n");
                for move in curr.path:
                    f.write(move[0]+ "\t"+move[1] + "\t" + str(move[2]) + "\t" + str(move[3]) + "\t" + str(move[4])+"\n")

                f.write("\n" + str(curr.board) + "\n")
                f.close()
                
                f = open("Results\\all_results.txt", "a")
                f.write(str(self.count) + "\t\t\tGBFS\t\th" + str(heuristic) + "\t\t" + str(curr.cost) + "\t\t\t\t" + str(statessearched) + "\t\t\t\t" + str("%5.2f" % runtime) + "\n")
                f.close
                
                return

            #If it's not a solution insert into closed list and increment counter
            self.closed.append(curr)
            statessearched+=1
        
            #print()
            for c in curr.board.cars:
                paths = curr.board.findCarPaths(c)    
                if len(paths) > 0:  
                    #print("Car ", c.letter, " can move:", paths)
                    
                    for i in range(len(paths)):
                        #Create a new node for successor
                        newnode = HNode(Board(curr.board.cars), 0, curr.cost)   
                        newnode.parent = curr
                        newnode.path = copy.deepcopy(newnode.parent.path)
                
                        #Apply the move and store it
                        move = newnode.board.moveCar(c.letter, paths[i])
                        newnode.path.append(move)
                        gCost = newnode.cost
                        newnode.addHeuristic(heuristic)
                        newnode.search = str(newnode.cost) + " " + str(gCost) + " " + str(newnode.cost - gCost) + " " + move[4] 
                    
                        #If the node is already in the open or closed list, skip it
                        if newnode in self.opened or newnode in self.closed:
                            continue

                        #Insert into open list (in sorted order)
                        bisect.insort(self.opened, newnode)

# Building State Space Search - Algorithm A or A* (A/A*)

In [61]:
class AAStar(object):
    def __init__(self):
        self.opened = []
        self.closed = []
        self.count = 0
        
    def next(self):
        self.count += 1
    
    def recursivePrinting(self, node):
        if node.cost != 0:
            self.recursivePrinting(node.parent)  
        print(node.board)
        
    def search(self, board, heuristic):
        #Initialise first Node with cost 0
        a = HNode(copy.deepcopy(board), 0, -1)
        self.opened.append(a)
        a.addHeuristic(heuristic)
        a.search = str(a.cost) + " 0 " + str(a.cost) + " " + a.board.stringLine()
        
        start_time = time.time()
        statessearched = 0
        
        f = open("Results\\a-h"+str(heuristic)+"-search-"+ str(self.count) +".txt", "w")
      
        while(1):              
            #If open list is empty, exit
            if(len(self.opened)==0):
                print("No Solution")
                #Close last file
                f.close()
                #Write everything to file
                f = open("Results\\a-h"+str(heuristic)+"-sol-"+ str(self.count) +".txt", "w")
                f.write("--------------------------------------------------------------------------------\n")
                f.write("Initial board configuration: " + board.stringLine())
                f.write("\n\n!\n" + str(board) + "\n")
                f.write("Car Fuel Available: ")
                for i in range(len(board.cars)):
                    f.write(""+board.cars[i].letter+":"+str(board.cars[i].fuel))
                    if i < len(board.cars) - 1:
                        f.write(", ")
                f.write("\n\n Sorry, could not solve the puzzle as specified.\nError: no solution found\n")
                runtime = time.time() - start_time
                f.write("Runtime: " + str(runtime) + " seconds")
                f.write("--------------------------------------------------------------------------------")
                f.close()
                
                f = open("Results\\all_results.txt", "a")
                f.write(str(self.count) + "\t\t\tA/A*\t\th" + str(heuristic) + "\t\tNA\t\t\t\t" + str(statessearched) + "\t\t\t\t" + str("%5.2f" % runtime) + "\n")
                f.close
                
                return
        
            #Pop the best candidate node
            curr = self.opened.pop(0)
            curr.setCost(curr.parentCost + 1)
            
            #print(curr.search)
            f.write(curr.search + "\n")
        
            #If the popped node is the solution, exit out and print stuff (later to file)
            if(curr.board.winConditionMet()):
                print("Initial board configuration")
                print()
                print(board)
                print("Car Fuel Available: ", end = '')
                for i in range(len(board.cars)):
                    print(""+board.cars[i].letter+":"+str(board.cars[i].fuel)+", ", end ='')
                print()
            
                print("Runtime: %s seconds" % (time.time() - start_time))
                print("Number of moves: ", curr.cost)
                print("Search path length: %s states" % statessearched)

                print("------")
                print("Solution path: ")
                for move in curr.path:
                    print(move[0]+ " "+move[1] + " " + str(move[2])+"; ", end='')
                    
                ######
                
                #Close last file
                f.close()
                #Write everything to file
                f = open("Results\\a-h"+str(heuristic)+"-sol-"+ str(self.count) +".txt", "w")
                f.write("Initial board configuration: " + board.stringLine())
                f.write("\n\n" + str(board) + "\n")
                f.write("Car Fuel Available: ")
                for i in range(len(board.cars)):
                    f.write(""+board.cars[i].letter+":"+str(board.cars[i].fuel))
                    if i < len(board.cars) - 1:
                        f.write(", ")
                f.write("\n\n")
                runtime = time.time() - start_time
                f.write("Runtime: " + str(runtime) + " seconds")
                f.write("\nNumber of moves: " + str(curr.cost))
                f.write("\nSearch path length: " + str(statessearched) + " states")

                # Recursive here....
                f.write("\nSolution path: ")
                for move in curr.path:
                    f.write(move[0]+ " "+move[1] + " " + str(move[2])+"; ")

                f.write("\n\n");
                for move in curr.path:
                    f.write(move[0]+ "\t"+move[1] + "\t" + str(move[2]) + "\t" + str(move[3]) + "\t" + str(move[4])+"\n")
                f.write("\n" + str(curr.board) + "\n")
                f.close()
                
                f = open("Results\\all_results.txt", "a")
                f.write(str(self.count) + "\t\t\tA\A*\t\th" + str(heuristic) + "\t\t" + str(curr.cost) + "\t\t\t\t" + str(statessearched) + "\t\t\t\t" + str("%5.2f" % runtime) + "\n")
                f.close
                
                return

            #If it's not a solution insert into closed list and increment counter
            self.closed.append(curr)
            statessearched+=1
        
            #print()
            for c in curr.board.cars:
                paths = curr.board.findCarPaths(c)    
                if len(paths) > 0:  
                    #print("Car ", c.letter, " can move:", paths)
                    
                    for i in range(len(paths)):
                        #Create a new node for successor
                        newnode = HNode(Board(curr.board.cars), curr.cost+1, curr.cost)   
                        newnode.parent = curr
                        newnode.path = copy.deepcopy(newnode.parent.path)
                
                        #Apply the move and store it
                        move = newnode.board.moveCar(c.letter, paths[i])
                        newnode.path.append(move)
                        gCost = newnode.cost
                        newnode.addHeuristic(heuristic)
                        newnode.search = str(newnode.cost) + " " + str(gCost) + " " + str(newnode.cost - gCost) + " " + move[4]      
                    
                        #If the node is already in the open or closed list, skip if the cost in is higher, else replace
                        if newnode in self.opened:
                            for n in self.opened:
                                if n == newnode and n.cost > newnode.cost:
                                    self.opened.remove(n)
                                    bisect.insort(self.opened, newnode)
                                    break
                            continue
                        elif newnode in self.closed:
                            for n in self.closed:
                                if n == newnode and n.cost > newnode.cost:
                                    self.closed.remove(n)
                                    bisect.insort(self.opened, newnode)
                                    break
                            continue
                        else:
                            #Insert into open list (in sorted order)
                            bisect.insort(self.opened, newnode)

# Outputs

In [76]:
currentBoard = Board(boards[49].cars)
ucs = UCS()
ucs.search(currentBoard)

Initial board configuration

..O.ZZ
..O.BB
AAO..C
.....C
...DEE
...D..

Car Fuel Available: O:100, Z:100, B:100, A:4, C:100, D:100, E:100, 
Runtime: 24.452993869781494 seconds
Number of moves:  5
Search path length: 1269 states
------
Solution path: 
O down 3; Z left 1; B left 1; C up 2; A right 4; 

In [77]:
currentBoard = Board(boards[49].cars)
gbfs = GBFS()
gbfs.search(currentBoard, 4)

Initial board configuration

..O.ZZ
..O.BB
AAO..C
.....C
...DEE
...D..

Car Fuel Available: O:100, Z:100, B:100, A:4, C:100, D:100, E:100, 
Runtime: 0.11099958419799805 seconds
Number of moves:  5
Search path length: 14 states
------
Solution path: 
O down 3; Z left 1; B left 1; C up 2; A right 4; 

In [78]:
currentBoard = Board(boards[1].cars)
aastar = AAStar()
aastar.search(currentBoard, 1)

Initial board configuration

....ZZ
....O.
.AA.OB
....OB
.....C
...DDC

Car Fuel Available: Z:100, O:100, A:100, B:100, C:100, D:100, 
Runtime: 4.045010805130005 seconds
Number of moves:  5
Search path length: 406 states
------
Solution path: 
Z left 1; B up 2; D left 1; O down 2; A right 3; 

In [62]:
f = open("Results\\all_results.txt", "w")
f.write("Puzzle number\tAlgorithm\tHeuristic\tLength of solution\tLength of search path\tExecution Time (seconds)\n")
f.close()

In [ ]:
ucs = UCS()

for b in boards:
    f = open("Results\\all_results.txt", "w")
    f.write("\n")
    f.close()
    currentBoard = Board(b.cars) 
    ucs.search(currentBoard)

In [ ]:
gbfs = GBFS()

for b in boards:
    f = open("Results\\all_results.txt", "w")
    f.write("\n")
    f.close()
    currentBoard = Board(b.cars) 
    gbfs.next()
    for i in range(1,5): 
        gbfs.search(currentBoard, i)

In [ ]:
aastar = AAStar()

for b in boards:
    f = open("Results\\all_results.txt", "w")
    f.write("\n")
    f.close()
    currentBoard = Board(b.cars) 
    aastar.next()
    for i in range(1,5): 
        aastar.search(currentBoard, i)